<a href="https://colab.research.google.com/github/otamajakusi/doc-vector/blob/main/sentence-transformers/bert_ldcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!git clone https://github.com/otamajakusi/doc-vector

Cloning into 'doc-vector'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 38 (delta 13), reused 31 (delta 9), pack-reused 0
Unpacking objects: 100% (38/38), 12.76 KiB | 1004.00 KiB/s, done.


In [18]:
%cd doc-vector/sentence-transformers

/content/doc-vector/sentence-transformers


In [22]:
!pip install -r requirements.txt
!pip install -r requirements-notebook.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.3 MB/s eta 0:00:00


In [20]:
!bash download-ldcc.sh

--2023-04-22 06:30:30--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.60MB/s    in 3.3s    

2023-04-22 06:30:34 (2.60 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [26]:
!python3 create_dataset.py --train-txt /content/drive/MyDrive/ml-bert/ldcc_train.txt --val-txt /content/drive/MyDrive/ml-bert/ldcc_val.txt --test-txt /content/drive/MyDrive/ml-bert/ldcc_text.txt

In [31]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
Unpacking objects: 100% (4/4), 362 bytes | 362.00 KiB/s, done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
From https://github.com/otamajakusi/doc-vector
   4e19cd7..fd80254  main       -> origin/main
Updating 4e19cd7..fd80254
Fast-forward
 sentence-transformers/bert_train.py | 2 ++
 1 file changed, 2 insertions(+)


In [32]:
!python3 bert_train.py --train-txt /content/drive/MyDrive/ml-bert/ldcc_train.txt --val-txt /content/drive/MyDrive/ml-bert/ldcc_val.txt --test-txt /content/drive/MyDrive/ml-bert/ldcc_text.txt --output-dir /content/drive/MyDrive/ml-bert

2023-04-22 07:17:02.958018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
label2id={'topic-news': 0, 'smax': 1, 'sports-watch': 2, 'peachy': 3, 'dokujo-tsushin': 4, 'movie-enter': 5, 'livedoor-homme': 6, 'it-life-hack': 7, 'kaden-channel': 8}
Using cuda device
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

In [33]:
!cp /drive/MyDrive/ml-bert/* /content/drive/MyDrive/ml-bert/

In [46]:
from transformers import BertForSequenceClassification
from transformers import BertJapaneseTokenizer
import torch.nn.functional as F
import json
from pathlib import Path
import random
import torch

label2id_path = "/content/drive/MyDrive/ml-bert/label2id.json"
label2id = json.loads(Path(label2id_path).read_text())
id2label = {value: key for key, value in label2id.items()}
print(label2id)

model_path = "/content/drive/MyDrive/ml-bert/model_save"
model = BertForSequenceClassification.from_pretrained(model_path)

tokenizer = BertJapaneseTokenizer.from_pretrained(model_path)

test_dataset_path =  "/content/drive/MyDrive/ml-bert/ldcc_text.txt"

with open(test_dataset_path) as f:
    test_dataset = f.read().split("\n")

test_label, test_text = random.choice(test_dataset).split("\t")
print(f"{test_label}:\n{test_text}")

encoded_input = tokenizer(
    test_text, max_length=512, padding=True, truncation=True, return_tensors="pt"
)
output = model(**encoded_input)
logits = output.logits
# sigmoid関数を適用して確率に変換する
probs = F.sigmoid(logits)
print(probs)
max_prob_index = torch.argmax(probs, dim=1)
print(max_prob_index)
print(id2label[max_prob_index.item()])

{'topic-news': 0, 'smax': 1, 'sports-watch': 2, 'peachy': 3, 'dokujo-tsushin': 4, 'movie-enter': 5, 'livedoor-homme': 6, 'it-life-hack': 7, 'kaden-channel': 8}
dokujo-tsushin:
http://news.livedoor.com/article/detail/6800344/ 2012-07-28T10:30:00+0900 昔、不幸の手紙は今やチェーンメール かつて「不幸の手紙」なるものがあった。ネットのない時代だったので、葉書に「この手紙を〇人に送らないと災いが訪れる」と書かれていて、現在の中高年世代の人たちは実際に指示された人数分の葉書を送ったそうだ。 それが今やチェーンメールとして姿を変え、その範囲は広がり加速度もついた。子どもたちの世界ではチェーンメールが横行し、「殺される」という内容に怯えた子どもの母親からの相談も警察に寄せられているという。  さすがに独女世代は、「チェーンメールは無視するもの」という概念が浸透していると思う。しかし、みゆきさん(29歳)は震災直後に届いた「地震で大きな被害を受けなかった西日本の関西電力で働いている友人からの情報」だという節電を求める内容に「これをできる限り広めてほしい」と付け加えられたメールを転送してしまったという。送ってきたのが親しい友人だったので、何の疑いも持たなかったそうだが、その送信者の友人もチェーンメールとは全く気付かなかったそうだ。  節電を求める内容は正しいことだが、そのために何人もの人にメールを送ればネットワークに負荷をかけたり、誤った情報が広まったりする。そんなことは普段なら分かっていても、非常事態では冷静な判断は難しい。実際、このメールは国内でかなり広まってしまったようだ。  この他にも、震災後は善意の募金を呼び掛けるチェーンメールが急増した。数々の復興募金などはスルーしていたのに、ついひっかかってしまったというのがゆかりさん(30歳)。  「パリに音楽留学をしていた同級生から『帰国して復興コンサートをするので、できるだけ多くの人に知らせてほしい』というメールが来たんです。場所は未定でしたが日時は記載されていましたので、無視するの